In [3]:
import pandas as pd  
import pickle  
import string  
import nltk  
from nltk.corpus import stopwords  
from nltk.stem.porter import PorterStemmer  
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score  
from sklearn.svm import SVC  

# Download NLTK stopwords  
nltk.download('stopwords')  
nltk.download('punkt')  

# Load dataset  
df = pd.read_csv("spam.csv", encoding="latin-1")  

# Keep only relevant columns  
df = df[['v1', 'v2']]  
df.columns = ['label', 'text']  

# Convert labels (ham = 0, spam = 1)  
df['label'] = df['label'].map({'ham': 0, 'spam': 1})  

# Text Preprocessing Function  
ps = PorterStemmer()  

def preprocess_text(text):  
    text = text.lower()  
    text = nltk.word_tokenize(text)  
    text = [word for word in text if word.isalnum()]  
    text = [word for word in text if word not in stopwords.words('english') and word not in string.punctuation]  
    text = [ps.stem(word) for word in text]  
    return " ".join(text)  

# Apply preprocessing  
df['text'] = df['text'].apply(preprocess_text)  

# Splitting Data  
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)  

# Vectorization (TF-IDF)  
vectorizer = TfidfVectorizer()  
X_train_tfidf = vectorizer.fit_transform(X_train)  
X_test_tfidf = vectorizer.transform(X_test)  

# Train SVM Model  
model_svm = SVC(kernel='linear', random_state=42)  
model_svm.fit(X_train_tfidf, y_train)  

# Evaluate Model  
y_pred = model_svm.predict(X_test_tfidf)  
accuracy = accuracy_score(y_test, y_pred)  
print(f"✅ SVM Model Accuracy: {accuracy:.2f}")  

# Save SVM model  
with open("model_svm.pkl", "wb") as f:  
    pickle.dump(model_svm, f)  

# Save vectorizer  
with open("vectorizer.pkl", "wb") as f:  
    pickle.dump(vectorizer, f)  

print("✅ SVM Model and Vectorizer Saved!")  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harikiran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harikiran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ SVM Model Accuracy: 0.98
✅ SVM Model and Vectorizer Saved!
